In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
if not (os.path.exists("fused-ffn-triton") or os.getcwd().endswith("fused-ffn-triton")):
  !git clone https://github.com/edtallison/fused-ffn-triton.git
else:
  print("already cloned")

already cloned


In [3]:
%cd fused-ffn-triton/

/content/fused-ffn-triton


In [4]:
!git pull origin main

From https://github.com/edtallison/fused-ffn-triton
 * branch            main       -> FETCH_HEAD
Already up to date.


In [5]:
!pip install -r requirements.txt

In [6]:
from experiments.benchmark import benchmark

In [7]:
benchmark()

Pytorch baseline: 0.3092s
Triton kernel: 2.2225s


In [8]:
!sed -n '1,20p' experiments/benchmark.py


import torch
import time

from torch_baseline.fused_block import FusedBlockTorch
from triton_kernel.driver import fused_block_triton

def benchmark():
    torch.manual_seed(0)
    M, N = 128, 4096
    device = "cuda"

    x = torch.randn((M, N), device=device)
    w1 = torch.randn((N, N), device=device)
    w2 = torch.randn((N, N), device=device)

    # Pytorch baseline
    model = FusedBlockTorch(N).to(device)
    model.linear1.weight.data.copy_(w1)
    model.linear1.bias.data.zero_()
    model.linear2.weight.data.copy_(w2)
